In [1]:
import pandas as pd
import numpy as np
from utils import *
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader

%load_ext autoreload
%autoreload 2

In [2]:
train_data, test_data = get_data()

In [3]:
test_data

,match_id,season,venue,innings,ball,striker,non_striker,bowler,runs_off_bat,extras,...,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,wicket,wickets_left,total_runs,runs_left
127,1304076,2022,"Brabourne Stadium, Mumbai",2,0.1,AJ Finch,SP Narine,TA Boult,0,0,...,0.0,0.0,run out,SP Narine,NaN,NaN,1,10,0,218.0
128,1304076,2022,"Brabourne Stadium, Mumbai",2,0.2,SS Iyer,AJ Finch,TA Boult,4,0,...,0.0,0.0,0,NaN,NaN,NaN,0,9,4,218.0
129,1304076,2022,"Brabourne Stadium, Mumbai",2,0.3,SS Iyer,AJ Finch,TA Boult,4,0,...,0.0,0.0,0,NaN,NaN,NaN,0,9,4,214.0
130,1304076,2022,"Brabourne Stadium, Mumbai",2,0.4,SS Iyer,AJ Finch,TA Boult,0,0,...,0.0,0.0,0,NaN,NaN,NaN,0,9,0,210.0
131,1304076,2022,"Brabourne Stadium, Mumbai",2,0.5,SS Iyer,AJ Finch,TA Boult,0,0,...,0.0,0.0,0,NaN,NaN,NaN,0,9,0,210.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,1304053,2022,"Brabourne Stadium, Mumbai",2,19.1,A Badoni,E Lewis,Mukesh Choudhary,0,1,...,0.0,0.0,0,NaN,NaN,NaN,0,6,1,9.0
248,1304053,2022,"Brabourne Stadium, Mumbai",2,19.2,A Badoni,E Lewis,Mukesh Choudhary,0,1,...,0.0,0.0,0,NaN,NaN,NaN,0,6,1,8.0
249,1304053,2022,"Brabourne Stadium, Mumbai",2,19.3,A Badoni,E Lewis,Mukesh Choudhary,6,0,...,0.0,0.0,0,NaN,NaN,NaN,0,6,6,7.0
250,1304053,2022,"Brabourne Stadium, Mumbai",2,19.4,A Badoni,E Lewis,Mukesh Choudhary,0,0,...,0.0,0.0,0,NaN,NaN,NaN,0,6,0,1.0


### Venue Encoding

In [4]:
enc = OneHotEncoder(sparse = False,handle_unknown='ignore')
train_data[[f'venue_{no}' for no in range(train_data.venue.nunique())]] = enc.fit_transform(train_data[['venue']])

/Users/rithickumar/miniforge3/envs/cs680/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [5]:
train_data

,match_id,season,venue,innings,ball,striker,non_striker,bowler,runs_off_bat,extras,...,venue_36,venue_37,venue_38,venue_39,venue_40,venue_41,venue_42,venue_43,venue_44,venue_45
50,598068,2013,M Chinnaswamy Stadium,2,0.1,MEK Hussey,M Vijay,R Rampaul,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,598068,2013,M Chinnaswamy Stadium,2,0.2,MEK Hussey,M Vijay,R Rampaul,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52,598068,2013,M Chinnaswamy Stadium,2,0.3,M Vijay,MEK Hussey,R Rampaul,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53,598068,2013,M Chinnaswamy Stadium,2,0.4,MEK Hussey,M Vijay,R Rampaul,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54,598068,2013,M Chinnaswamy Stadium,2,0.5,MEK Hussey,M Vijay,R Rampaul,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,598065,2013,"Rajiv Gandhi International Stadium, Uppal",2,19.2,KK Cooper,PV Tambe,DW Steyn,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
241,598065,2013,"Rajiv Gandhi International Stadium, Uppal",2,19.3,KK Cooper,PV Tambe,DW Steyn,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
242,598065,2013,"Rajiv Gandhi International Stadium, Uppal",2,19.4,KK Cooper,PV Tambe,DW Steyn,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
243,598065,2013,"Rajiv Gandhi International Stadium, Uppal",2,19.5,PV Tambe,SK Trivedi,DW Steyn,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
train_cols = ['innings', 'ball','wickets_left','runs_left']
train_cols.extend([f'venue_{no}' for no in range(train_data.venue.nunique())])
target_cols = ['runs_off_bat']

In [7]:
test_data[[f'venue_{no}' for no in range(train_data.venue.nunique())]] = enc.transform(test_data[['venue']])

## Naive Model

In [8]:
train_data.loc[:,'over'] = train_data['ball']//1
test_data.loc[:,'over'] = test_data['ball']//1

In [9]:
run_means = train_data.groupby(['innings','venue','over'])['runs_off_bat'].mean()

In [10]:
run_means_overall = train_data.groupby(['innings','over'])['runs_off_bat'].mean()
# run_means_overall[2][1]

In [11]:
run_means[2]['Arun Jaitley Stadium']

over
0.0     1.082353
1.0     1.352941
2.0     1.702381
3.0     1.500000
4.0     1.252874
5.0     1.720930
6.0     1.069767
7.0     1.069767
8.0     1.139535
9.0     1.295455
10.0    1.302326
11.0    1.229885
12.0    1.317647
13.0    0.875000
14.0    1.234568
15.0    1.287500
16.0    1.328767
17.0    1.116883
18.0    1.342857
19.0    1.346939
Name: runs_off_bat, dtype: float64

In [12]:
baseline_preds = []
for ven,over in zip(test_data.venue,test_data.over):
    try:
        baseline_preds.append(run_means[2][ven][over])
    except:
        baseline_preds.append(run_means_overall[2][over])

# baseline_preds = [run_means[2][ven][over] for ven,over in zip(test_data.venue,test_data.over)]

In [13]:
mean_squared_error(test_data[target_cols],baseline_preds,squared=True)

2.9743247454898984

## XGBoost

In [14]:
xgb_model = XGBRegressor()

In [15]:
xgb_model.fit(train_data[train_cols],train_data[target_cols])

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [16]:
mean_squared_error(train_data[target_cols],xgb_model.predict(train_data[train_cols]),squared=True)

2.3369571657336503

In [17]:
mean_squared_error(test_data[target_cols],xgb_model.predict(test_data[train_cols]),squared=True)

2.9293723386981756

## Linear Model

In [18]:
lin_model = LinearRegression()

In [19]:
lin_model.fit(train_data[train_cols],train_data[target_cols])

LinearRegression()

In [20]:
mean_squared_error(train_data[target_cols],lin_model.predict(train_data[train_cols]),squared=True)

2.5490193911284242

In [21]:
mean_squared_error(test_data[target_cols],lin_model.predict(test_data[train_cols]),squared=True)

1.684893863440402e+17

## Proposed Model

In [22]:
# lstm_train_cols = ['innings', 'ball', 'wicket','runs_off_bat','wickets_left','runs_left']
lstm_train_cols = ['total_runs','wicket','wickets_left','runs_left']
state_train_cols = ['innings','ball','wickets_left','runs_left']
state_train_cols.extend([f'venue_{no}' for no in range(train_data.venue.nunique())])

In [23]:
lookback = 6

In [24]:
lstm_train,current_state_train,target_train = prep_lstm_dataset(train_data,lstm_train_cols,state_train_cols,target_cols,lookback=6)

In [25]:
train_dataset = custom_dataset(np.vstack(lstm_train),np.vstack(current_state_train),np.vstack(target_train))

In [37]:
train_loader = DataLoader(train_dataset,batch_size=len(train_dataset),shuffle=True)

In [38]:
device = torch.device('cpu')

In [39]:
epochs = 500

In [40]:
lstm_train[0].shape

(50, 6, 4)

In [41]:
pressure_model = xR_Model(state_input_size=50,lstm_input_size=len(lstm_train_cols),lstm_hidden_size=1,seq_len=6)
pressure_model.to(device)

xR_Model(
  (pressure_layer): LSTM(4, 1, batch_first=True)
  (pressure_linear): Linear(in_features=6, out_features=1, bias=True)
  (comb_layer): Linear(in_features=51, out_features=1, bias=True)
)

In [42]:
loss_reg = torch.nn.MSELoss()
# loss_class = torch.nn.BCELoss()
optimizer = torch.optim.Adam(pressure_model.parameters())

In [43]:
for epoch in range(epochs):
    epoch_loss = 0
    # torch.nn.init.xavier_uniform_(model.pressure_layer.all_weights,gain= 'sigmoid')
    for batch,data in enumerate(train_loader):
        
        optimizer.zero_grad()
        lstm_x,state_x,target = data
        lstm_x = lstm_x.to(device)
        state_x = state_x.to(device)

        target = target.to(device)

        reg_out,lstm_output = pressure_model(lstm_x,state_x)

        reg_out = reg_out.flatten()
        # class_out = class_out.flatten()

        loss_1 = loss_reg(reg_out,target[:,0])
 
        loss_1.backward()

        optimizer.step()

        epoch_loss += loss_1.item()

        # if batch %1000 == 0:
        #     # print(f" Epoch {epoch+1} batch {batch+1} : reg_loss {loss_1.item()}  class_loss {loss_2.item()}")
        #     print(f" Epoch {epoch+1} batch {batch+1} : reg_loss {loss_1.item()} ")
    print(epoch_loss)


3.3770508766174316
3.3312976360321045
3.2831978797912598
3.232853412628174
3.179612636566162
3.1217000484466553
3.0564427375793457
2.983067274093628
2.909108877182007
2.8518786430358887
2.820650815963745
2.8138113021850586
2.8245489597320557
2.8425114154815674
2.8574371337890625
2.862729549407959
2.8565874099731445
2.8408191204071045
2.81906795501709
2.795412063598633
2.7734827995300293
2.7559280395507812
2.744112253189087
2.7380542755126953
2.7365570068359375
2.7376177310943604
2.7390172481536865
2.7389121055603027
2.7362382411956787
2.7308056354522705
2.723172187805176
2.714364528656006
2.705561637878418
2.697794198989868
2.69173526763916
2.6875898838043213
2.6851089000701904
2.6837222576141357
2.6827142238616943
2.6814329624176025
2.679450035095215
2.6766374111175537
2.673151731491089
2.669344425201416
2.665632963180542
2.6623761653900146
2.6597847938537598
2.657883405685425
2.656529188156128
2.6554770469665527
2.6544597148895264
2.6532716751098633
2.6518125534057617
2.6500999927520

In [45]:
pressure_model

xR_Model(
  (pressure_layer): LSTM(4, 1, batch_first=True)
  (pressure_linear): Linear(in_features=6, out_features=1, bias=True)
  (comb_layer): Linear(in_features=51, out_features=1, bias=True)
)

In [46]:
pressure_model.eval()
for batch,data in enumerate(train_loader):
        lstm_x,state_x,target = data
        lstm_x = lstm_x.to(device)
        state_x = state_x.to(device)

        target = target.to(device)
        reg_out,lstm_output = pressure_model(lstm_x,state_x)
        reg_out = reg_out.flatten()

        print(mean_squared_error(train_data[target_cols],reg_out.detach().numpy()))

2.604958460764218


In [47]:
lstm_test,current_state_test,target_test = prep_lstm_dataset(test_data,lstm_train_cols,state_train_cols,target_cols,lookback=lookback)
test_dataset = custom_dataset(np.vstack(lstm_test),np.vstack(current_state_test),np.vstack(target_test))
testloader = DataLoader(test_dataset,batch_size=len(test_dataset),shuffle=False)


In [48]:
pressure_model.eval()
for batch,data in enumerate(testloader):
        lstm_x,state_x,target = data
        lstm_x = lstm_x.to(device)
        state_x = state_x.to(device)

        target = target.to(device)
        reg_out,lstm_output = pressure_model(lstm_x,state_x)
        reg_out = reg_out.flatten()

        print(mean_squared_error(test_data[target_cols],reg_out.detach().numpy()))

2.8805522808906208


### Inference

In [49]:
def debug(model,dataloader):

    p_i = []
    x_r = []
    model.eval()

    for batch,data in enumerate(dataloader):
            
            # data = data.to(device)
            lstm_x,state_x,target = data
            lstm_x = lstm_x.to(device)
            state_x = state_x.to(device)
            # target = target.type(torch.LongTensor)
            # print("Target :",target.shape)
            target = target.to(device)

            reg_out,lstm_output = model(lstm_x,state_x)
            reg_out = reg_out.flatten()
            # class_out = class_out.flatten()
            lstm_output = lstm_output.flatten()

            p_i.extend(lstm_output.tolist())
            x_r.extend(reg_out.tolist())
    
#     plt.hist(x_r)
    
    return p_i,x_r

In [50]:
p_i , x_r = debug(pressure_model,train_loader)

In [51]:
debug_data = train_data.copy()
debug_data['Pressure'] = np.array(p_i)
debug_data['xR'] = x_r
debug_data.query("match_id == 598067")[['ball','runs_left','wickets_left','Pressure','total_runs','wicket','runs_off_bat','xR']]

,ball,runs_left,wickets_left,Pressure,total_runs,wicket,runs_off_bat,xR
123,0.1,173.0,10,0.956579,5,0,0,1.436808
124,0.2,168.0,10,0.103515,1,0,1,1.151367
125,0.3,167.0,10,0.339667,1,0,0,1.063510
126,0.4,166.0,10,0.573273,1,0,1,1.210552
127,0.5,165.0,10,0.495866,4,0,4,1.091237
...,...,...,...,...,...,...,...,...
245,19.2,45.0,2,0.653022,1,0,1,1.365948
246,19.3,44.0,2,0.262995,4,0,4,0.976406
247,19.4,40.0,2,0.673863,0,0,0,1.177095
248,19.5,40.0,2,0.914947,0,1,0,1.208399


In [52]:
p_i , x_r = debug(pressure_model,testloader)
debug_data = test_data.copy()
debug_data['Pressure'] = np.array(p_i)
debug_data['xR'] = x_r
debug_data.query("match_id == 1359487")[['ball','runs_left','wickets_left','Pressure','total_runs','wicket','runs_off_bat','xR']].tail(10)

,ball,runs_left,wickets_left,Pressure,total_runs,wicket,runs_off_bat,xR
240,18.5,39.0,3,0.115582,0,0,0,1.314812
241,18.6,39.0,3,0.113488,0,0,0,1.319593
242,18.7,39.0,3,0.108285,6,0,6,1.323744
243,18.8,33.0,3,0.103962,4,0,4,1.317991
244,19.1,29.0,3,0.126135,1,0,1,1.331381
245,19.2,28.0,3,0.234281,6,0,6,1.356828
246,19.3,22.0,3,0.458553,6,0,6,1.397413
247,19.4,16.0,3,0.793646,6,0,6,1.460462
248,19.5,10.0,3,0.947748,6,0,6,1.486823
249,19.6,4.0,3,0.939650,6,0,6,1.480304


In [53]:
debug_data.query("match_id == 1370350")[['ball','runs_left','wickets_left','Pressure','total_runs','wicket','runs_off_bat','xR']].tail(10)

,ball,runs_left,wickets_left,Pressure,total_runs,wicket,runs_off_bat,xR
244,18.5,31.0,1,0.115763,4,0,4,1.006465
245,18.6,27.0,1,0.102663,0,0,0,1.002293
246,19.1,27.0,1,0.255896,1,0,1,1.059384
247,19.2,26.0,1,0.091429,2,0,0,1.029570
248,19.3,24.0,1,0.098204,1,0,1,1.032788
249,19.4,23.0,1,0.125818,1,0,1,1.041911
250,19.5,22.0,1,0.300330,4,0,4,1.080812
251,19.6,18.0,1,0.105118,1,0,1,1.039724
252,19.7,17.0,1,0.151074,1,0,0,1.052565
253,19.8,16.0,1,0.215908,0,1,0,1.069233


In [54]:
torch.save(pressure_model.state_dict(), "pressure_model_v2.pt")

In [ ]:
# v2 - 4 statecols v1 - 2 state cols

In [ ]:
https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/chennai-super-kings-vs-gujarat-titans-qualifier-1-1370350/full-scorecard